<a href="https://colab.research.google.com/github/merucode/RL/blob/93-Project-Trader-Custom_enviromnet/02_Reinforce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# To Do List

# Install Dependency and Test

In [ ]:
!git clone -b 93-Project-Trader-Custom_enviromnet https://github.com/merucode/RL.git
!cd RL && mv * ../

In [ ]:
!pip install gym==0.25.2
!pip install -e gym-examples

### NOTE: After inistall gym-examples, Restart runtime

# STEP. Ready Data

In [3]:
import pandas as pd

df1 = pd.read_csv('KRW-BTC_5m_171001_200903_upbit.csv')
df2 = pd.read_csv('KRW-BTC_5m_200904_230810_upbit.csv')

df = pd.concat([df1, df2], ignore_index=True)
df = df[df.columns[1:]]
print(df.tail(3))

              open        high         low       close    volume
492909  39190000.0  39231000.0  39190000.0  39212000.0  2.929709
492910  39212000.0  39214000.0  39188000.0  39191000.0  3.407502
492911  39191000.0  39210000.0  39188000.0  39188000.0  3.884740


In [4]:
# Preporcessing df_render
#df = df[:400000] #Temp : TEST
df_ohlc = df[df.columns[:4]] / 10000
df_v = df[df.columns[-1:]]
df = pd.concat([df_ohlc, df_v], axis=1)
df_render = df
print(df_render)

          open    high     low   close    volume
0        479.7   480.0   479.3   480.0  0.013444
1        479.8   479.9   479.2   479.3  0.013597
2        479.5   480.2   479.5   479.5  0.016304
3        480.1   480.2   479.5   480.1  0.017136
4        479.7   479.9   479.1   479.9  0.007615
...        ...     ...     ...     ...       ...
492907  3920.5  3920.8  3917.9  3920.6  2.508436
492908  3920.6  3921.2  3918.6  3921.2  1.350777
492909  3919.0  3923.1  3919.0  3921.2  2.929709
492910  3921.2  3921.4  3918.8  3919.1  3.407502
492911  3919.1  3921.0  3918.8  3918.8  3.884740

[492912 rows x 5 columns]


In [ ]:
# Preporcessing df
df_ohlc = df[df.columns[:4]]
df_ohlc_normalization = (df_ohlc - df_ohlc.mean())/df_ohlc.std()
df_v = df[df.columns[-1:]]
df_v_normalization =  (df_v - df_v.mean())/df_v.std()
df = pd.concat([df_ohlc_normalization, df_v_normalization], axis=1)
print(df)

### Check env

In [ ]:
import time
import matplotlib.pyplot as plt
import gym
import numpy as np

In [ ]:
env = gym.vector.make("gym_examples:TradeWorld-v0", num_envs=2, df=df, df_render=df_render, obs_len=2016)

print(f'observation space        : {env.observation_space}')
print(f"observation single space : {env.single_observation_space.shape[0]}")
print(f'action space             : {env.action_space}')
print(f'action single shape      : {env.single_action_space.n}')

In [ ]:
obs, info = env.reset(return_info=True)
print(obs)
print(obs.shape)
print(info)

In [ ]:
actions = env.action_space.sample()
print(actions)
env.step(actions)

# STEP. Reinforce_multi

In [ ]:
pip install pytorch-lightning==2.0.6

In [5]:
import copy
import torch
import random
import gym
import matplotlib

import numpy as np
import matplotlib.pyplot as plt

import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics, \
  NormalizeObservation, NormalizeReward

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\lightning_fabric\__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\torchmetrics\utilities\imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\torchvision\transforms\_functional_pil.py:242: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
c:\Users\k

In [13]:
class GradientPolicy(nn.Module):

    def __init__(self, in_features, n_actions, hidden_size=1260):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2, hidden_size*1)
        self.fc4 = nn.Linear(hidden_size*1, n_actions)

    def forward(self, x):
        x = torch.tensor(x).float().to(device)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=-1)
        return x

policy = GradientPolicy(10080, 16)#GradientPolicy(env.single_observation_space.shape[0], env.single_action_space.n)
policy

GradientPolicy(
  (fc1): Linear(in_features=10080, out_features=5040, bias=True)
  (fc2): Linear(in_features=5040, out_features=2520, bias=True)
  (fc3): Linear(in_features=2520, out_features=1260, bias=True)
  (fc4): Linear(in_features=1260, out_features=16, bias=True)
)

In [7]:
def create_env(env_name, num_envs, **kwargs):
    env = gym.vector.make(env_name, num_envs=num_envs, **kwargs)
    env = RecordEpisodeStatistics(env)
    return env

In [8]:
class RLDataset(IterableDataset):

    def __init__(self, env, policy, steps_per_epoch, gamma):
        self.env = env
        self.policy = policy
        self.steps_per_epoch = steps_per_epoch
        self.gamma = gamma
        self.obs = env.reset()

    @torch.no_grad()
    def __iter__(self):
        transitions = []

        for step in range(self.steps_per_epoch):
            action = self.policy(self.obs)
            action = action.multinomial(1).cpu().numpy()
            next_obs, reward, done, info = self.env.step(action.flatten())
            transitions.append((self.obs, action, reward, done))
            self.obs = next_obs

        obs_b, action_b, reward_b, done_b = map(np.stack, zip(*transitions))

        running_return = np.zeros(self.env.num_envs, dtype=np.float32)
        return_b = np.zeros_like(reward_b)

        for row in range(self.steps_per_epoch - 1, -1, -1):
            running_return = reward_b[row] + (1 - done_b[row]) * self.gamma * running_return
            return_b[row] = running_return

        num_samples = self.env.num_envs * self.steps_per_epoch
        obs_b = obs_b.reshape(num_samples, -1)
        action_b = action_b.reshape(num_samples, -1)
        return_b = return_b.reshape(num_samples, -1)

        idx = list(range(num_samples))
        random.shuffle(idx)

        for i in idx:
            yield obs_b[i], action_b[i], return_b[i]

In [15]:
class Reinforce(LightningModule):

    def __init__(self, env_name, num_envs=8, samples_per_epoch=10, batch_size=2048,
                hidden_size=1060, policy_lr=0.001, gamma=0.99, entropy_coef=0.001, optim=AdamW):

        super().__init__()
        self.training_step_outputs = []
        self.env = create_env(env_name, num_envs=num_envs, df=df, df_render=df_render, obs_len=2016)

        obs_size = self.env.single_observation_space.shape[0]
        n_actions = self.env.single_action_space.n

        self.policy = GradientPolicy(obs_size, n_actions, hidden_size)
        self.dataset = RLDataset(self.env, self.policy, samples_per_epoch, gamma)

        self.save_hyperparameters()
        self.training_step_outputs = []


    # Configure optimizers.
    def configure_optimizers(self):
        return self.hparams.optim(self.policy.parameters(), lr=self.hparams.policy_lr)

    def train_dataloader(self):
        return DataLoader(dataset=self.dataset, batch_size=self.hparams.batch_size)

    # Training step.
    def training_step(self, batch, batch_idx):
        obs, actions, returns = batch
        
        probs = self.policy(obs)
        log_probs = torch.log(probs + 1e-6)
        action_log_prob = log_probs.gather(1, actions)

        entropy = - torch.sum(probs * log_probs, dim=-1, keepdim=True)

        pg_loss = - action_log_prob * returns
        loss = (pg_loss - self.hparams.entropy_coef * entropy).mean()

        self.log("episode/PG Loss", pg_loss.mean())
        self.log("episode/Entropy", entropy.mean())
        #self.log("episode/returns", returns.mean())
        self.training_step_outputs.append(loss)

        return loss

    def on_train_epoch_end(self):
         epoch_average = torch.stack(self.training_step_outputs).mean()
         self.log("training_epoch_average", epoch_average)
         self.training_step_outputs.clear()  # free memory

In [11]:
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'rm'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


Reusing TensorBoard on port 6006 (pid 16480), started 1:39:48 ago. (Use '!kill 16480' to kill it.)

In [17]:
algo = Reinforce('gym_examples:TradeWorld-v0')

trainer = Trainer(
  devices=num_gpus,
  accelerator="gpu",
  max_epochs=300,
  log_every_n_steps=1
)

trainer.fit(algo)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type           | Params
------------------------------------------
0 | policy | GradientPolicy | 54.0 M
------------------------------------------
54.0 M    Trainable params
0         Non-trainable params
54.0 M    Total params
215.998   Total estimated model params size (MB)


Epoch 0: : 1it [00:00,  3.11it/s, v_num=9]

C:\Users\kym92\AppData\Local\Temp\ipykernel_2032\3842473335.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).float().to(device)


Epoch 299: : 1it [00:00,  3.79it/s, v_num=9]

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: : 1it [00:02,  2.26s/it, v_num=9]


In [18]:
torch.save(algo.policy.state_dict(), 'model_weights.pth')

# STEP. Render

In [19]:
from torch.distributions import Categorical

def model_render(env_name, **kwargs):
    env = gym.make(env_name, **kwargs)
    episode_trigger=lambda e: True  # all Episode Recode

    env = gym.wrappers.RecordVideo(env, "./video", episode_trigger=episode_trigger, video_length=100000)

    for episode in range(1):
        s = env.reset()
        done = False
        score = 0

        pi = GradientPolicy(10080, 16, hidden_size=1060)
        pi.load_state_dict(torch.load('model_weights.pth'))
        pi.eval()

        pi.to(device)

        while not done:
            prob = pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample()
            s_prime, r, done, info = env.step(a.item())
            done_mask = 0.0 if done else 1.0
            s = s_prime
            score += r
            #print(f"DEBUG balance: {info['balance']}, {a.item()}")

        print(f"# of episode {episode}:, avg score : {score}")
    env.close()

In [20]:
model_render("gym_examples:TradeWorld-v0", df=df, df_render=df_render, obs_len=2016)

Epoch 231: : 1it [15:32, 932.71s/it, v_num=8]


c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\gym\wrappers\record_video.py:78: UserWarning: WARN: Overwriting existing videos at c:\Users\kym92\project\trader\02_custom_environment\video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\gym\utils\passive_env_checker.py:141: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float64, actual type: float32
  logger.warn(
c:\Users\kym92\anaconda3\envs\trade_world\lib\site-packages\gym\wrappers\monitoring\video_recorder.py:7

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('./video/rl-video-episode-0.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=500 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Test Code